In [3]:
using Pkg
using Random, Distributions
using PyPlot
using Optim
Pkg.add("DataFrames")
using DataFrames
Pkg.add("CSV")
using CSV

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
 Installed WeakRefStrings ─ v0.6.2
 Installed FilePathsBase ── v0.7.0
 Installed CSV ──────────── v0.6.0
  Updating `~/.julia/environments/v1.0/Project.toml`
  [336ed68f] + CSV v0.6.0
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [336ed68f] + CSV v0.6.0
  [48062228] + FilePathsBase v0.7.0
  [ea10d353] + WeakRefStrings v0.6.2


┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1192
┌ Warning: Module Parsers with build ID 809019240130 is missing from the cache.
│ This may mean Parsers [69de0a69-1ddd-5017-9359-2bf0b02dc9f0] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947
┌ Info: Precompiling WeakRefStrings [ea10d353-3f73-51f8-a26c-33c1cb351aa5]
└ @ Base loading.jl:1192
┌ Info: Precompiling FilePathsBase [48062228-2e41-5def-b9a4-89aafe57970f]
└ @ Base loading.jl:1192


In [6]:
data = CSV.read("monte_carlo.csv");
head(data)

MethodError: MethodError: no method matching Parsers.Options(::Missing, ::UInt8, ::UInt8, ::UInt8, ::UInt8, ::UInt8, ::UInt8, ::UInt8, ::Nothing, ::Nothing, ::Nothing, ::Bool, ::Bool, ::Nothing, ::Bool, ::Bool, ::Bool, ::Bool)
Closest candidates are:
  Parsers.Options(::Union{Missing, Nothing, Array{String,1}}, ::Union{Char, UInt8}, ::Union{Char, UInt8}, ::Union{Char, UInt8}, ::Union{Char, UInt8}, ::Union{Char, UInt8}, ::Union{Nothing, Char, UInt8, String}, ::Union{Char, UInt8}, ::Union{Nothing, Array{String,1}}, ::Union{Nothing, Array{String,1}}, ::Union{Nothing, String, DateFormat}, ::Any, ::Any, ::Any, ::Any, ::Any) at /home/erichschulman/.julia/packages/Parsers/GLY4Q/src/Parsers.jl:60
  Parsers.Options(::Union{Missing, Nothing, Array{String,1}}, ::Union{Char, UInt8}, ::Union{Char, UInt8}, ::Union{Char, UInt8}, ::Union{Char, UInt8}, ::Union{Char, UInt8}, ::Union{Nothing, Char, UInt8, String}, ::Union{Char, UInt8}, ::Union{Nothing, Array{String,1}}, ::Union{Nothing, Array{String,1}},

In [ ]:
function ll_obj(params)
    n = self.exog.shape[0]
    k = int(self.exog.shape[1]/2)

    #calculate the utility from each action
    u1 = np.dot(self.exog[:,0:k], params[0:k])  + params[k]*self.endog[:,1]
    u2 = np.dot(self.exog[:,k:], params[k+1:2*k+1]) + params[2*k+1]*self.endog[:,0]
        
    #calculate the log-likelihood
    ll = self.endog[:,0]*(u1) + self.endog[:,1]*(u2)
    ll = ll - np.log(1.+np.exp(u1)) - np.log(1.+np.exp(u2))
                                      
    #adjust for eq selection
    mult1 = self.endog[:,0]*(1-self.endog[:,1])
    mult = self.endog[:,0]*(1-self.endog[:,1]) + self.endog[:,1]*(1-self.endog[:,0])
        
    ll = mult1*params[2*k+2] - mult*np.log( 1+ np.exp(params[2*k+2]) )
    print(ll.sum()/n)
        
    return - ll.sum()/n
end